## **Import Libraries**

In [ ]:
#data-manipulation
import pandas as pd
import numpy as np
import nltk

In [ ]:
#dtata-visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#financial-data & black litterman model
!pip install yfinance
!pip install PyPortfolioOpt
import yfinance as yf
from pypfopt import black_litterman, risk_models
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 958 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 3.4 MB/s 


# **Test Space**

In [ ]:
#identify risk-free rate to find implied returns using ^IRX (13 Week T-Bill)
price_df = yf.download(
    ['^IRX','AAPL','BBY','BAC','SBUX','T'],
    start='2017-01-01',
    end='2020-01-01',
    interval='1mo').dropna().pct_change()[1:]

[*********************100%***********************]  6 of 6 completed


# **Black Letterman Inputs**

In [ ]:
#identify risk-free rate to find implied returns using ^IRX (13 Week T-Bill)
def risk_free(start_date, end_date):
    rfdf = yf.download('^IRX',start=start_date,end=end_date,interval='1d').dropna()
    return rfdf['Adj Close']

In [ ]:
 def market_data(start_date,end_date):
        market_df = yf.download('^GSPC',start=start_date,end=end_date,interval='1d').dropna()
        return market_df['Adj Close']

In [ ]:
def get_historicals(start_date,stock_tickers,end_date,data_type='Adj Close',returns=True):
    """ 
        Pulls the historical data from a specified start_date and end_date. 
        start_date: '20xx-MM-DD'
        data_type: lets you choose between Open, Close, AdjClose
        returns: returuns the pct_change data frame
        stock_tickers: list of stock_tickers interest
    """
    if returns:
        historicals = yf.download(
            ['^IRX']+stock_tickers,
            start=start_date,
            end=end_date,
            interval='1d'
        ).dropna()
    else:
        historicals = yf.download(
            ['^IRX']+stock_tickers,
            start=start_date,
            end=end_date,
            interval='1d'
        ).dropna().pct_change().dropna()[1:]
        
    return historicals[data_type]

In [ ]:
def risk_premium(returns_df,cov = True):
    temp = pd.DataFrame()
    for i in returns_df.columns:
        temp[i] = returns_df['^IRX']
    premium = returns_df - temp
    if cov:
        return premium.cov().iloc[0:-1,:-1]
    else:
        return premium.iloc[0:-1,:-1]

In [ ]:
def market_cap(stock_tickers):
    market_cap = {}
    for t in stock_tickers:
        stock = yf.Ticker(t)
        market_cap[t] = stock.info["marketCap"]
    return market_cap

In [ ]:
def QandPMatrix(views,stock_ticker):
    #Assume it's in format: [[('MSFT','>','GE'),0.02],['MSFT',0.02]]
    q_matrix = []
    p_matrix = np.zeros((len(views),len(stock_ticker)))
    for view in views:
        q_matrix.append(view[1])
    for i in range(len(views)):
        if type(views[i][0]) is str:
            pos = stock_ticker.index(views[i][0]) 
            p_matrix[i][pos] = 1 
        else:
            pos_1 = stock_ticker.index(views[i][0][0])
            pos_2 = stock_ticker.index(views[i][0][2])
            if views[i][0][1] == '>':
                p_matrix[i][pos_1] = 1
                p_matrix[i][pos_2] = -1
            else:
                p_matrix[i][pos_1] = -1
                p_matrix[i][pos_2] = 1
    return [q_matrix,p_matrix]

In [ ]:
def RunBlackLittermanModel(start_date, end_date,stock_tickers,views):
    delta = black_litterman.market_implied_risk_aversion(
        market_data(start_date,end_date),
        frequency=252,
        risk_free_rate= risk_free(start_date, end_date)
    ).mean()
    returns_df = get_historicals(start_date,stock_tickers,end_date)
    cov_matrix = risk_premium(returns_df,cov = True)
    prior = black_litterman.market_implied_prior_returns(
        market_cap(stock_tickers), 
        delta, 
        cov_matrix
      )
    tau = 0.05
    QandP = QandPMatrix(views,stock_tickers)
    Q = np.transpose(QandP[0])
    P = QandP[1]
    omega = tau * np.dot(np.dot(P,risk_premium(returns_df,cov = True)),np.transpose(P))
    bl = BlackLittermanModel(
        cov_matrix = cov_matrix,
        pi = prior,
        Q = Q,
        P = P,
        omega = omega,
        tau = tau
    )  
    bl.bl_weights(delta)
    bl_weights = dict(bl.clean_weights())
    bl_rets = bl.bl_returns()
    return (bl_weights,bl_rets)

# RunBlackLittermanModel

In [ ]:
RunBlackLittermanModel(
    '2020-01-01',
    '2021-01-01',
    ['AAPL','AMZN','CCL','NCLH'],
    [[('AAPL','>','CCL'),1],['AMZN',0.30],[('AMZN','>','NCLH'),1]]
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  5 of 5 completed


({'AAPL': 0.55933, 'AMZN': 0.29661, 'CCL': 0.53838, 'NCLH': -0.39432},
 AAPL   -282.753378
 AMZN   -349.340432
 CCL      48.666349
 NCLH     51.035503
 dtype: float64)